In [36]:
import torch
import numpy as np
import onnxruntime as ort
from stable_baselines3 import SAC

In [37]:
torch_model = SAC.load('./../assets/g1-forward')

In [38]:
input = np.array([0.0] * 119).reshape(1, 119).astype(np.float32)

In [51]:
torch_mean = np.zeros(12)

n_iter = 10000
for i in range(n_iter):
    torch_mean += torch_model.policy.predict(torch.from_numpy(input).float())[0][0]

torch_mean/n_iter
torch_mean

array([-5232.02768734, -9256.49316448, -9769.03249478,  4742.5439958 ,
        4766.3663854 , -9727.51090693, -7289.71801066, -4803.99115235,
       -9639.31192291,  7165.20577157, -4364.83300203, -9530.80311477])

In [52]:
torch_model.policy.action_space

Box([-0.802851 -1.0472   -2.69653  -0.802851 -1.0472   -2.69653  -0.802851
 -1.0472   -2.69653  -0.802851 -1.0472   -2.69653 ], [ 0.802851  4.18879  -0.916298  0.802851  4.18879  -0.916298  0.802851
  4.18879  -0.916298  0.802851  4.18879  -0.916298], (12,), float32)

In [ ]:
def scale_action(scaled_action ,action_space):
    low, high = action_space.low, action_space.high
    return low + (0.5 * (scaled_action + 1.0) * (high - low))

In [46]:
onnx_mean = np.zeros(12)
ort_sess = ort.InferenceSession('./../assets/g1-forward.onnx')

n_iter = 10000
for i in range(n_iter):
    onnx_mean += ort_sess.run(None, {'obs': input})[0][0]

onnx_mean/n_iter
onnx_mean

array([-6514.36762797, -9523.70335982,  9328.56398707,  5927.71452494,
       -4158.10327213,  9337.60048291, -9094.07935127, -7836.99624672,
        9462.4126415 ,  8929.80290843, -7671.65321006,  9574.82129016])